In [ ]:
import os
import random
import subprocess
from multiprocessing import Pool
import time
import pandas as pd
# Generate a list of path of filtered MSA
path_decipher = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"
path_files = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/15122022_session"

all_proteins = pd.read_csv(f"{path_files}/part_III_ptB/all_prophage_proteins.names.db.fasta", names = ["proteins"])

paths = [f"{path_decipher}/{prot.split('__')[0]}/mmseqs_out/{prot.split('__')[1]}/{prot.split('.fasta')[0]}.MSA.a2m" for prot in all_proteins["proteins"] 
         if os.path.isfile(f"{path_decipher}/{prot.split('__')[0]}/mmseqs_out/{prot.split('__')[1]}/{prot.split('.fasta')[0]}.MSA.a2m")==True 
         if os.path.isfile(f"{path_decipher}/{prot.split('__')[0]}/DBsuite_depo/{prot.split('__')[1]}/{prot.split('.fasta')[0]}.suite.hhr")==False]


def scan_depolymerase(path_query) :
    path_db = "/home/conchae/databases/depolymerase_building/DBsuite_depolymerase/depolymerase_db.suite"
    phage = path_query.split("__")[1]
    query = path_query.split("/")[-1].split(".MSA")[0]
    path_dir_short = f"{'/'.join(path_query.split('/')[0:-3])}/DBsuite_depo"
    path_dir = f"{'/'.join(path_query.split('/')[0:-3])}/DBsuite_depo/{phage}"
    try :
        os.mkdir(path_dir_short)
    except FileExistsError :
        pass
    try :
        os.mkdir(path_dir)
    except FileExistsError :
        pass
    path_out = f"{path_dir}/{query}"
    hhmscan_cmmd = f"hhsearch -i {path_query} -d  {path_db} -o {path_out}.suite.hhr -blasttab {path_out}.suite.tab"
    hhmscan_process = subprocess.Popen(hhmscan_cmmd, shell =True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    scan_out, scan_err = hhmscan_process.communicate()
    with open(f"{path_files}/hhsearch_done","a+") as outfile :
        outfile.write(f"{path_query}\n")

if __name__ == '__main__':
    with Pool(30) as p:
        p.map(scan_depolymerase, paths)
        

In [ ]:
import os
#import random
#import subprocess
#from tqdm import tqdm
import time
import pandas as pd
#import pprint
import json 
#pp = pprint.PrettyPrinter(width = 150)
# Generate a list of path of filtered MSA 
path_decipher = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"
path_db = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/15122022_session"

out_names = ["query","target","x1","hmm","mismatch","gapopen","query_start","query_stop","target_start","target_stop","evalue","score"]

paths = [f"{path_decipher}/{prot.split('__')[0]}/mmseqs_out/{prot.split('__')[1]}/{prot.split('.fasta')[0]}.MSA.a2m" for prot in all_proteins["proteins"][0:20] 
         if os.path.isfile(f"{path_decipher}/{prot.split('__')[0]}/mmseqs_out/{prot.split('__')[1]}/{prot.split('.fasta')[0]}.MSA.a2m")==True]


def scan_depolymerase(path_query) :
    path_db = "/home/conchae/databases/depolymerase_building/DBsuite_Dpo.v3/DBsuite_Dpo.v3"
    phage = path_query.split("__")[1]
    query = path_query.split("/")[-1].split(".MSA")[0]
    path_dir_short = f"{'/'.join(path_query.split('/')[0:-3])}/DBsuite_depo_v3"
    path_dir = f"{'/'.join(path_query.split('/')[0:-3])}/DBsuite_depo_v3/{phage}"
    try :
        os.mkdir(path_dir_short)
    except FileExistsError :
        pass
    try :
        os.mkdir(path_dir)
    except FileExistsError :
        pass
    path_out = f"{path_dir}/{query}"
    hhmscan_cmmd = f"hhsearch -i {path_query} -d  {path_db} -o {path_out}.suite.hhr -blasttab {path_out}.suite.tab"
    hhmscan_process = subprocess.Popen(hhmscan_cmmd, shell =True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    scan_out, scan_err = hhmscan_process.communicate()
    print(path_query , scan_out , scan_err , sep = "\n")
    with open(f"{path_files}/hhsearch_done_v3","a+") as outfile :
        outfile.write(f"{path_query}\n")


scores_out , dico_count_out, dico_ipr_out, depolymerase_dico_out = scan_tab(path_tabs["Paths"])

with open(f"{path_db}/dico_ipr_out.v3.json", "w") as outfile:
    json.dump(dico_ipr_out, outfile)
with open(f"{path_db}/dico_ipr_count.v3.json", "w") as outfile:
    json.dump(dico_count_out, outfile)    
with open(f"{path_db}/dbsuite_results.v3.json", "w") as outfile:
    json.dump(depolymerase_dico_out, outfile) 
    
scores_list = pd.DataFrame(scores_out)
scores_list.to_csv(f"{path_db}/scores.dbsuite.v3.txt", header=None, index=False)